# Step 2: Private Prediction using Syft Keras - Serving (Server)

In [1]:
from collections import OrderedDict
#import torch
import tensorflow as tf
import numpy as np
import syft
import tf_encrypted as tfe

In [2]:
#import tf_encrypted as tfe
from syft.keras.model import Sequential
from syft.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

## Protocol

In [3]:
players = OrderedDict([
    ('server0', 'localhost:4000'),
    ('server1', 'localhost:4001'),
    ('server2', 'localhost:4002'),
])

config = tfe.RemoteConfig(players)
config.save('/tmp/tfe.config')

In [4]:
tfe.set_config(config)
tfe.set_protocol(tfe.protocol.SecureNN())

## Launching servers

Before actually serving the computation below we need to launch TensorFlow servers in new processes. Run the following in three different terminals. You may have to allow Python to accept incoming connections.

In [5]:
for player_name in players.keys():
    print("python -m tf_encrypted.player --config /tmp/tfe.config {}".format(player_name))

python -m tf_encrypted.player --config /tmp/tfe.config server0
python -m tf_encrypted.player --config /tmp/tfe.config server1
python -m tf_encrypted.player --config /tmp/tfe.config server2


## Model

In [6]:
task_classes = 10
task_shape = [1, 28, 28, 1]
pre_trained_weights = 'short-conv-mnist.h5'

In [7]:
model = Sequential()

model.add(Conv2D(10, (3, 3), batch_size=1, input_shape=task_shape[1:]))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(task_classes, name="logit"))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [8]:
# Load pre-trained weights
model.load_weights(pre_trained_weights)

In [9]:
data = np.ones([1, 28, 28, 1])

In [10]:
public_prediction = model.predict(data)
print("Public Prediction:", public_prediction)

Public Prediction: [[  2.9898617  -3.8340132   6.4398613   4.636752   -4.7098727   0.3969848
    7.697638  -15.947172    7.6116843  -3.8501363]]


### Secure the model by sharing the weights

In [11]:
prot = tfe.protocol.SecureNN()

tfe_model = model.share(prot=prot)

INFO:tf_encrypted:Performing an activation before a pooling layer can result in unnecessary performance loss. Check model definition in case of missed optimization.


seq__init__ <tf_encrypted.protocol.securenn.securenn.SecureNN object at 0x147c8f358>


INFO:tf_encrypted:Performing an activation before a pooling layer can result in unnecessary performance loss. Check model definition in case of missed optimization.
INFO:tf_encrypted:Performing an activation before a pooling layer can result in unnecessary performance loss. Check model definition in case of missed optimization.


## Computation

**Note computation, tfe.serving.QueueServer etc. will move into model.share()**

In [12]:
input_shape = (1, 28, 28, 1)
output_shape = (1, 10)

def computation(x):
    return tfe_model(x)

In [13]:
server = tfe.serving.QueueServer(
    input_shape=input_shape,
    output_shape=output_shape,
    computation_fn=computation)

In [14]:
sess = tfe.Session(disable_optimizations=True)

tf.Session.reset(sess.target)

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
  optimizer_options {
    opt_level: L0
  }
  rewrite_options {
    arithmetic_optimization: OFF
  }
}



### Serving

In [ ]:
def step_fn():
    print("Next")

sess.run(tf.global_variables_initializer(), tag='init')

server.run(
    sess,
    num_steps=5,
    step_fn=step_fn)

Next
